# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

In [94]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

## Откройте и изучите файл

In [95]:
df = pd.read_csv('/datasets/users_behavior.csv')
df.sample(5)

,calls,minutes,messages,mb_used,is_ultra
2942,70.0,426.37,0.0,18780.23,0
2673,53.0,378.17,24.0,7136.39,0
1572,13.0,81.38,10.0,5024.19,1
1285,64.0,432.25,14.0,20009.42,0
62,93.0,622.66,0.0,20894.56,1


In [96]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [97]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


Медиана почти равна среднему, данные  не имеют выбросов

In [98]:
df.duplicated().sum()

0

Дубликатов нет

## Разбейте данные на выборки

In [99]:
df_train, df_t = train_test_split(df, test_size=0.35, random_state=12345)
df_test, df_final = train_test_split(df_t, test_size=0.5, random_state=12345)
df_train.shape, df_test.shape, df_final.shape

((2089, 5), (562, 5), (563, 5))

In [100]:
df_train_feature = df_train.drop('is_ultra', axis=1)
df_train_target = df_train['is_ultra']

df_test_feature = df_test.drop('is_ultra', axis=1)
df_test_target = df_test['is_ultra']

df_final_feature = df_final.drop('is_ultra', axis=1)
df_final_target = df_final['is_ultra']

## Исследуйте модели

### DecisionTreeClassifier

In [101]:
best_accuracy = 0
best_depth = 0
for depth in range(2, 15):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(df_train_feature, df_train_target)
    predictions = model.predict(df_test_feature)
    accuracy = accuracy_score(df_test_target, predictions)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_depth = depth
        model_decision_tree = model
        print(f"accuracy_score train: {accuracy_score(df_train_target, model.predict(df_train_feature))}")
        print(f"accuracy_score test: {accuracy_score(df_test_target, model.predict(df_test_feature))}")

best_accuracy, depth

accuracy_score train: 0.7865007180469124
accuracy_score test: 0.791814946619217
accuracy_score train: 0.8046912398276688
accuracy_score test: 0.797153024911032


(0.797153024911032, 14)

Для дерева решений получили метрику качества accuracy = 0.79, при max_depth = 14, это не плохо, но есть и другие модели.

### RandomForestClassifier

In [102]:
best_accuracy = 0
best_params = []
for depth in range(2, 10):
    for est in range(2, 30):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model.fit(df_train_feature, df_train_target)
        predictions = model.predict(df_test_feature)
        accuracy = accuracy_score(df_test_target, predictions)
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_params = [depth, est]
            model_random_forest = model
            print(f"accuracy_score train: {accuracy_score(df_train_target, model.predict(df_train_feature))}")
            print(f"accuracy_score test: {accuracy_score(df_test_target, model.predict(df_test_feature))}")
            
best_accuracy, best_params

accuracy_score train: 0.7855433221637147
accuracy_score test: 0.791814946619217
accuracy_score train: 0.7826711345141216
accuracy_score test: 0.800711743772242
accuracy_score train: 0.8123504068932503
accuracy_score test: 0.802491103202847
accuracy_score train: 0.8219243657252274
accuracy_score test: 0.8042704626334519
accuracy_score train: 0.8319770224988032
accuracy_score test: 0.806049822064057
accuracy_score train: 0.8334131163235998
accuracy_score test: 0.8078291814946619
accuracy_score train: 0.842508377213978
accuracy_score test: 0.8096085409252669
accuracy_score train: 0.8501675442795597
accuracy_score test: 0.8113879003558719
accuracy_score train: 0.8544758257539492
accuracy_score test: 0.8131672597864769
accuracy_score train: 0.852082336045955
accuracy_score test: 0.8149466192170819
accuracy_score train: 0.8693154619435136
accuracy_score test: 0.8167259786476868


(0.8167259786476868, [8, 23])

На тренировочном датасете показатель стал лучше - 0.81 при n_estimators = 23, а max_depth = 8

### LogisticRegression

In [103]:
best_accuracy = 0
reg = 0
for reg in ['l1', 'l2']:
    model = LogisticRegression(penalty=reg, random_state=12345)
    model.fit(df_train_feature, df_train_target)
    predictions = model.predict(df_test_feature)
    accuracy = accuracy_score(df_test_target, predictions)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        reg = reg
        model_logistic_regression = model
        print(f"accuracy_score train: {accuracy_score(df_train_target, model.predict(df_train_feature))}")
        print(f"accuracy_score test: {accuracy_score(df_test_target, model.predict(df_test_feature))}")
            
best_accuracy, reg

accuracy_score train: 0.7496409765438009
accuracy_score test: 0.7597864768683275


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


(0.7597864768683275, 'l2')

Логистическая регрессия показала лучший скор при l2 регуляризации = 0.75, но тем не менее этого недостаточно.

Таким образом лучшая модель - модель случайного леса при n_estimators = 23, а max_depth = 8. При этом мы также видим некоторое переобучение, так как accuracy_score train: 0.8693154619435136 превышает accuracy_score test: 0.8167259786476868

## Проверьте модель на тестовой выборке

In [104]:
predictions = model_random_forest.predict(df_final_feature)
accuracy = accuracy_score(df_final_target, predictions)

In [105]:
accuracy

0.7904085257548845

Неплохой результат, но чувствуется переобучение

## (бонус) Проверьте модели на адекватность

In [106]:
df_final_feature.head()

,calls,minutes,messages,mb_used
2657,37.0,258.09,0.0,45180.75
1004,37.0,279.41,0.0,27548.01
2013,114.0,815.02,178.0,25104.76
2455,71.0,499.54,56.0,9791.01
3208,164.0,1016.98,71.0,17787.52


Допустим у нас все звонки были равны нулю:

In [107]:
df_final_feature['calls'] = 0

In [108]:
df_final_feature.sample(3)

,calls,minutes,messages,mb_used
3143,0,439.39,82.0,19315.86
655,0,688.70,16.0,18758.43
593,0,679.61,19.0,16174.21


Тогда попробуем предсказать на таких "порченных данных":

In [109]:
predictions = model_random_forest.predict(df_final_feature)
accuracy = accuracy_score(df_final_target, predictions)

In [110]:
accuracy

0.6642984014209592

Судя по тому, что модель все же смогла выдать 66% правильных ответов, значит мы не зря ее учили, что она может разобраться в частично выбросных данных и даже тут нашла закономерности.

## Вывод

- Мы разделили изначальный датасет на 3 датасета: 
    - обучались на 65% выборки
    - валидировались на половине от оставшейся выборки и опирались на метрику качества модели по ней
    - на оставшейся половине мы уже подводили итоги
- Использовали 3 модели:
    - DecisionTreeClassifier дал нам результат в 0.79
    - RandomForestClassifier допрыгнул до 0.81
    - LogisticRegression даже с l2 регуляризацией показала 0.75
- Провели финальный тест на датасете, итоговый скор - 0.79
- Проверили модель на прочность и закинули в нее выбросные значения для одного из столбцов, но модель проявила устойчивость к выбросам и на выходе даже дала 66% правильных ответов